In [1]:
from envs.minimal_jsp_env.util.jsp_generation.random_generator import RandomJSPGenerator, RandomJSPGeneratorPool, RandomJSPGeneratorOperationDistirbution, RandomJSPGeneratorWithJobPool
from envs.minimal_jsp_env.entities import Operation, JSPInstance
import json
from typing import List

In [2]:
import collections
import numpy as np
import pandas as pd

from ortools.sat.python import cp_model

###############################################################################
# OR job shop problem python, see https://developers.google.com/optimization/scheduling/job_shop
##############################################################################



def solve_jsp(jobs_data, max_time_in_seconds=7200, verbose=False):
    """Minimal jobshop problem."""
    # Create the model.
    model = cp_model.CpModel()

    machines_count = 1 + max(task[0] for job in jobs_data for task in job)
    all_machines = range(machines_count)

    # Computes horizon dynamically as the sum of all durations.
    horizon = sum(task[1] for job in jobs_data for task in job)

    # Named tuple to store information about created variables.
    task_type = collections.namedtuple('task_type', 'start end interval')
    # Named tuple to manipulate solution information.
    assigned_task_type = collections.namedtuple('assigned_task_type',
                                                'start job index duration')

    # Creates job intervals and add to the corresponding machine lists.
    all_tasks = {}
    machine_to_intervals = collections.defaultdict(list)

    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine = task[0]
            duration = task[1]
            suffix = '_%i_%i' % (job_id, task_id)
            start_var = model.NewIntVar(0, horizon, 'start' + suffix)
            end_var = model.NewIntVar(0, horizon, 'end' + suffix)
            interval_var = model.NewIntervalVar(start_var, duration, end_var,
                                                'interval' + suffix)
            all_tasks[job_id, task_id] = task_type(
                start=start_var, end=end_var, interval=interval_var)
            machine_to_intervals[machine].append(interval_var)

    # Create and add disjunctive constraints.
    for machine in all_machines:
        model.AddNoOverlap(machine_to_intervals[machine])

    # Precedences inside a job.
    for job_id, job in enumerate(jobs_data):
        for task_id in range(len(job) - 1):
            model.Add(all_tasks[job_id, task_id +
                                1].start >= all_tasks[job_id, task_id].end)

    # Makespan objective.
    obj_var = model.NewIntVar(0, horizon, 'makespan')
    model.AddMaxEquality(obj_var, [
        all_tasks[job_id, len(job) - 1].end
        for job_id, job in enumerate(jobs_data)
    ])
    model.Minimize(obj_var)

    # Solve model.
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = max_time_in_seconds
    status = solver.Solve(model)

    if (status == cp_model.OPTIMAL) or (status == cp_model.FEASIBLE):
        # Create one list of assigned tasks per machine.
        assigned_jobs = collections.defaultdict(list)
        for job_id, job in enumerate(jobs_data):
            for task_id, task in enumerate(job):
                machine = task[0]
                assigned_jobs[machine].append(
                    assigned_task_type(
                        start=solver.Value(all_tasks[job_id, task_id].start),
                        job=job_id,
                        index=task_id,
                        duration=task[1]))

        # Create per machine output lines.
        output = ''
        gantt_data = []

        for machine in all_machines:
            # Sort by starting time.
            assigned_jobs[machine].sort()
            sol_line_tasks = 'Machine ' + str(machine) + ': '
            sol_line = '           '

            for assigned_task in assigned_jobs[machine]:
                name = 'job_%i_%i' % (assigned_task.job, assigned_task.index)
                # Add spaces to output to align columns.
                sol_line_tasks += '%-10s' % name

                start = assigned_task.start
                duration = assigned_task.duration
                sol_tmp = '[%i,%i]' % (start, start + duration)
                # Add spaces to output to align columns.
                sol_line += '%-10s' % sol_tmp

                # add Job_num,Mach_num,Job_time,Start_time
                gantt_data.append([assigned_task.job, machine, duration, start])

            sol_line += '\n'
            sol_line_tasks += '\n'
            output += sol_line_tasks
            output += sol_line

        gantt_data = pd.DataFrame(data=gantt_data, columns=["Job_num", "Mach_num", "Job_time", "Start_time"])

        # Finally print the solution found.
        optimal_time = solver.ObjectiveValue()
        if verbose:
            print("################# OR Solver Solution #################")
            print('Optimal Schedule Length: %i' % optimal_time)
            print(output)
            print("#######################################################")
        status_dict = {4: 'OPTIMAL', 2: 'FEASIBLE'}
        return optimal_time, output, gantt_data, status_dict[status]


In [3]:
class JSPReaderGoogleOR:
    def read_instance(self, path: str) -> List:       

        with open(path, 'r') as file:
            input_file = json.load(file)

        jobs = []

        for job_machine_types, job_durations in zip(input_file['machine_types'], input_file['durations']):
            
            operations = []
            for operation_machine_type, operation_duration in zip(job_machine_types, job_durations):
                operations.append([operation_machine_type, operation_duration])
            
            jobs.append(operations)


        return jobs

In [4]:
test_instance = JSPReaderGoogleOR().read_instance('data/entropy_datasets/entropy0_4/entropy0_4_49')
test_instance

[[[1, 9], [1, 9], [1, 1], [1, 9], [1, 1], [1, 1]],
 [[1, 9], [1, 9], [1, 9], [1, 1], [1, 9], [1, 9]],
 [[1, 2], [1, 1], [1, 9], [1, 1], [1, 2], [1, 1]],
 [[1, 1], [4, 4], [4, 4], [1, 9], [4, 4], [1, 9]],
 [[1, 9], [1, 9], [1, 1], [4, 4], [1, 1], [1, 2]],
 [[4, 4], [1, 9], [1, 1], [4, 4], [1, 9], [1, 1]]]

In [5]:
optimal_time, output, gantt_data, status = solve_jsp(test_instance, verbose=True)

################# OR Solver Solution #################
Optimal Schedule Length: 153
Machine 0: 
           
Machine 1: job_3_0   job_2_0   job_2_1   job_0_0   job_0_1   job_0_2   job_0_3   job_0_4   job_0_5   job_1_0   job_1_1   job_1_2   job_1_3   job_1_4   job_1_5   job_2_2   job_2_3   job_2_4   job_2_5   job_3_3   job_4_0   job_3_5   job_5_1   job_5_2   job_4_1   job_4_2   job_5_4   job_4_4   job_5_5   job_4_5   
           [0,1]     [1,3]     [3,4]     [4,13]    [13,22]   [22,23]   [23,32]   [32,33]   [33,34]   [34,43]   [43,52]   [52,61]   [61,62]   [62,71]   [71,80]   [80,89]   [89,90]   [90,92]   [92,93]   [93,102]  [102,111] [111,120] [120,129] [129,130] [130,139] [139,140] [140,149] [149,150] [150,151] [151,153] 
Machine 2: 
           
Machine 3: 
           
Machine 4: job_5_0   job_3_1   job_3_2   job_3_4   job_5_3   job_4_3   
           [0,4]     [4,8]     [8,12]    [102,106] [130,134] [140,144] 

#######################################################


In [6]:
print(f"optimal_time={optimal_time}")
print(f"output={output}")
print(f"gannt data = {gantt_data}")
print(f"status={status}")

optimal_time=153.0
output=Machine 0: 
           
Machine 1: job_3_0   job_2_0   job_2_1   job_0_0   job_0_1   job_0_2   job_0_3   job_0_4   job_0_5   job_1_0   job_1_1   job_1_2   job_1_3   job_1_4   job_1_5   job_2_2   job_2_3   job_2_4   job_2_5   job_3_3   job_4_0   job_3_5   job_5_1   job_5_2   job_4_1   job_4_2   job_5_4   job_4_4   job_5_5   job_4_5   
           [0,1]     [1,3]     [3,4]     [4,13]    [13,22]   [22,23]   [23,32]   [32,33]   [33,34]   [34,43]   [43,52]   [52,61]   [61,62]   [62,71]   [71,80]   [80,89]   [89,90]   [90,92]   [92,93]   [93,102]  [102,111] [111,120] [120,129] [129,130] [130,139] [139,140] [140,149] [149,150] [150,151] [151,153] 
Machine 2: 
           
Machine 3: 
           
Machine 4: job_5_0   job_3_1   job_3_2   job_3_4   job_5_3   job_4_3   
           [0,4]     [4,8]     [8,12]    [102,106] [130,134] [140,144] 

gannt data =     Job_num  Mach_num  Job_time  Start_time
0         3         1         1           0
1         2         1         2 

In [7]:
from envs.minimal_jsp_env.util.jsp_conversion.readers import JSPReaderJSON

test_instance2 = JSPReaderJSON().read_instance('data/entropy_datasets/entropy0_4/entropy0_4_49')

In [8]:
from envs.minimal_jsp_env.util.jsp_solver import JSPSolver

In [9]:
JSPSolver().solve(test_instance2)

153.0